In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.lyrics.com/genres.php?genre='
genres = ['Pop', 'Hip%20Hop','Folk,%20World,%20__%20Country', 'Jazz', 'Rock']

In [3]:
def get_artist(div):
    return div.find_all('p', class_ = "lyric-meta-album-artist")[0].getText()

In [4]:
def get_lyrics_from_url(lyrics_url):
    r = requests.get('https://www.lyrics.com/' + lyrics_url).content
    soup = BeautifulSoup(r, "lxml")
    try : 
        l = soup.find_all('pre')[0].getText()
    except : 
        l = 'NaN'
    return l

In [5]:
def get_title_url(div):
    data = div.find_all('p', class_= "lyric-meta-title")[0]
    lyrics_url = data.find_all('a', href = True)[0]['href']
    title = data.getText()
    return title, lyrics_url

In [12]:
def get_artist_from_page(df, genre, nb):
    page_url = url + genre + '&p=' + str(nb)
    resp = requests.get(page_url).content
    soup = BeautifulSoup(resp, "lxml")
    content = soup.find_all('div', class_= "lyric-meta col-sm-6 col-xs-6")
    for c in content:
        t, l = get_title_url(c)
        df.loc[len(df)] = [t, get_artist(c), genre, l]
    return df

In [7]:
genres_emma = genres[:3]
genres_oli = genres[3:]
MIN_PAGES = 0
MAX_PAGES = 10

In [13]:
lyrics_df = pd.DataFrame(columns=['title','artist', 'genre', 'lyrics_url'])

In [14]:
for genre in genres_emma:
    for page in range(MIN_PAGES, MAX_PAGES):
        get_artist_from_page(lyrics_df, genre, page)
        print('finished genre : ', genre, ' page : ', page)

finished genre :  Pop  page :  0
finished genre :  Pop  page :  1
finished genre :  Pop  page :  2
finished genre :  Pop  page :  3
finished genre :  Pop  page :  4
finished genre :  Pop  page :  5
finished genre :  Pop  page :  6
finished genre :  Pop  page :  7
finished genre :  Pop  page :  8
finished genre :  Pop  page :  9
finished genre :  Hip%20Hop  page :  0
finished genre :  Hip%20Hop  page :  1
finished genre :  Hip%20Hop  page :  2
finished genre :  Hip%20Hop  page :  3
finished genre :  Hip%20Hop  page :  4
finished genre :  Hip%20Hop  page :  5
finished genre :  Hip%20Hop  page :  6
finished genre :  Hip%20Hop  page :  7
finished genre :  Hip%20Hop  page :  8
finished genre :  Hip%20Hop  page :  9
finished genre :  Folk,%20World,%20__%20Country  page :  0
finished genre :  Folk,%20World,%20__%20Country  page :  1
finished genre :  Folk,%20World,%20__%20Country  page :  2
finished genre :  Folk,%20World,%20__%20Country  page :  3
finished genre :  Folk,%20World,%20__%20Coun

In [16]:
lyrics_df = lyrics_df.drop_duplicates('title')

In [17]:
lyrics_df['lyrics'] = lyrics_df['lyrics_url'].map(lambda x : get_lyrics_from_url(x))

In [18]:
lyrics_df = lyrics_df.dropna(axis = 1)
lyrics_df = lyrics_df.drop_duplicates('lyrics')
lyrics_df.head(10)

,title,artist,genre,lyrics_url,lyrics
0,Little Things Mean a Lot,Antoinette,Pop,/lyric/31577731/Antoinette/Little+Things+Mean+...,Blow me a kiss from across the room\r\nSay I l...
1,Rhymin' and Stealin',Beastie Boys,Pop,/lyric/2567092/Beastie+Boys/Rhymin%27+and+Stea...,Because mutiny on the bounty's what we're all ...
4,Armada Latina,Cypress Hill,Pop,/lyric/19375506/Cypress+Hill/Armada+Latina,NaN
7,We Want the Funk,Gerardo,Pop,/lyric/1014258/Gerardo/We+Want+the+Funk,"Excuse me, where have I seen you? \r\nThey ten..."
9,Downtown,Kool Moe Dee,Pop,/lyric/32235126/Kool+Moe+Dee/Downtown,"I went to the Moped store, said fuck it\r\nSal..."
10,A Salt With a Deadly Pepa,Salt-N-Pepa,Pop,/lyric/940355/Salt-N-Pepa/A+Salt+With+a+Deadly...,I'm Salt here to resource so start steppin' \r...
11,I Like It Like That,Salt-N-Pepa,Pop,/lyric/940356/Salt-N-Pepa/I+Like+It+Like+That,[You want me to say it now? \r\n\r\n\r\n\r\nc'...
12,Spinderella's Not a Fella (But a Girl D.J.),Salt-N-Pepa,Pop,/lyric/940362/Salt-N-Pepa/Spinderella%27s+Not+...,When the needle's picked up the volume's kicke...
14,Book of Love,The Four Seasons,Pop,/lyric/2598/The+Four+Seasons/Book+of+Love,I write down\r\nEverything that you say\r\nSo ...
15,Love and Other Bruises,Air Supply,Pop,/lyric/3082/Air+Supply/Love+and+Other+Bruises,As the time slips through your fingers\r\nTill...


In [19]:
genre_dict = {'Pop': 'pop', 'Hip%20Hop' : 'hip hop','Folk,%20World,%20__%20Country': 'country', 'Jazz':'jazz', 'Rock':'rock'}

In [111]:
#Avant de quitter le notebook 
"""new_lyrics_df = lyrics_df.drop('lyrics_url', axis = 1)
new_lyrics_df['genre'] = new_lyrics_df['genre'].map(lambda x : genre_dict[x])
new_lyrics_df.to_csv("lyrics_com.csv", index = False)"""

,title,artist,genre,lyrics
0,Little Things Mean a Lot,Antoinette,Pop,Blow me a kiss from across the room\r\nSay I l...
1,Rhymin' and Stealin',Beastie Boys,Pop,Because mutiny on the bounty's what we're all ...
4,Armada Latina,Cypress Hill,Pop,NaN
7,We Want the Funk,Gerardo,Pop,"Excuse me, where have I seen you? \r\nThey ten..."
9,Downtown,Kool Moe Dee,Pop,"I went to the Moped store, said fuck it\r\nSal..."
10,A Salt With a Deadly Pepa,Salt-N-Pepa,Pop,I'm Salt here to resource so start steppin' \r...
11,I Like It Like That,Salt-N-Pepa,Pop,[You want me to say it now? \r\n\r\n\r\n\r\nc'...
12,Spinderella's Not a Fella (But a Girl D.J.),Salt-N-Pepa,Pop,When the needle's picked up the volume's kicke...
14,Book of Love,The Four Seasons,Pop,I write down\r\nEverything that you say\r\nSo ...
15,Love and Other Bruises,Air Supply,Pop,As the time slips through your fingers\r\nTill...
